# **画像パート　作業サーバーセットアップ編**

このNotebookは，情報科学演習で使う作業サーバーに各自の作業環境を準備する手順をまとめたものです．  
基本的には，以下の流れで進めます．

1. 各自のユーザを作成
2. SSH公開鍵認証を設定
3. conda環境の作成
4. GPU周りの設定
5. PyTorchのインストール
---

### **1. 各自のユーザを作成**
1. VSCodeの拡張機能Remote-SSHをインストール  
<img src="../images/remote_ssh.png" width="700"> 
<br>

2. SSH configを編集  
    1. 「ctrl + shift + P」でコマンドパレットを開く
    2. 「Remote SSH:ホストに接続する」を選択
    3. 「SSHホストを構成する…」を選択
    4. 「C:~\\.ssh\\config」を選択
    5. 下記画像の様にSSH設定を編集  
<img src="../images/ssh_config.png" width="700">
<br>

3. Remote-SSH を使って，作業サーバーのrootアカウントにSSH

4. 各学生のユーザ作成  
例: ユーザ名を `hoge` とします．
    1. ユーザ作成 & ホームディレクトリ作成．注)フルネームとか聞かれるが，エンター連打で大丈夫  
        <br>
        <div style="position: relative; background-color:#002b36; color:#ffffff; padding:10px; border-radius:5px; font-family:monospace;">
        <div style="position:absolute; top:-10px; left:10px; background:#268bd2; color:#fff; padding:2px 6px; border-radius:3px; font-size:12px; font-family:sans-serif;">
            bash
        </div>
        sudo adduser hoge<br>
        </div>
    2. ユーザ作成確認
        <br>
        <br>
        <div style="position: relative; background-color:#002b36; color:#ffffff; padding:10px; border-radius:5px; font-family:monospace;">
        <div style="position:absolute; top:-10px; left:10px; background:#268bd2; color:#fff; padding:2px 6px; border-radius:3px; font-size:12px; font-family:sans-serif;">
            bash
        </div>
        cat /etc/passwd | grep hoge<br>
        </div>
    2. sudo権限付与
        <br>
        <br>
        <div style="position: relative; background-color:#002b36; color:#ffffff; padding:10px; border-radius:5px; font-family:monospace;">
        <div style="position:absolute; top:-10px; left:10px; background:#268bd2; color:#fff; padding:2px 6px; border-radius:3px; font-size:12px; font-family:sans-serif;">
            bash
        </div>
        sudo gpasswd -a hoge sudo<br>
        </div>

5. Remote-SSH を使って，作業サーバーのuserアカウントにSSH

---

### **2. 公開鍵認証設定**
毎回パスワード入力するのは面倒なので，公開鍵認証を設定します．  
1. 公開鍵作成(Windows想定)
    1. コマンドプロンプトで公開鍵作成  
    .sshディレクトリにhoge_keyとhoge_key.pubが生成される
        <br>
        <br>
        <div style="position: relative; background-color:#002b36; color:#ffffff; padding:10px; border-radius:5px; font-family:monospace;">
        <div style="position:absolute; top:-10px; left:10px; background:#268bd2; color:#fff; padding:2px 6px; border-radius:3px; font-size:12px; font-family:sans-serif;">
            bash
        </div>
        ssh-keygen -f C:\~\.ssh\hoge_key<br>
        </div>
2. SSH configの編集  
configファイルの"IdentityFile"に先ほど作成した秘密鍵(.pubじゃない方)の絶対パスを追加
3. 作業サーバーに公開鍵を設定
    1. .sshディレクトリの作成
        <br>
        <br>
        <div style="position: relative; background-color:#002b36; color:#ffffff; padding:10px; border-radius:5px; font-family:monospace;">
        <div style="position:absolute; top:-10px; left:10px; background:#268bd2; color:#fff; padding:2px 6px; border-radius:3px; font-size:12px; font-family:sans-serif;">
            bash
        </div>
        mkdir -p .ssh<br>
        </div>
    2. authorized_keysの作成
        <br>
        <br>
        <div style="position: relative; background-color:#002b36; color:#ffffff; padding:10px; border-radius:5px; font-family:monospace;">
        <div style="position:absolute; top:-10px; left:10px; background:#268bd2; color:#fff; padding:2px 6px; border-radius:3px; font-size:12px; font-family:sans-serif;">
            bash
        </div>
        touch .ssh/authorized_keys<br>
        </div>
    3. 公開鍵のコピー
        <br>
        <br>
        <div style="position: relative; background-color:#002b36; color:#ffffff; padding:10px; border-radius:5px; font-family:monospace;">
        <div style="position:absolute; top:-10px; left:10px; background:#268bd2; color:#fff; padding:2px 6px; border-radius:3px; font-size:12px; font-family:sans-serif;">
            bash
        </div>
        # ノートPCのコマンドプロンプトで実行<br>
        scp C:~\.ssh\hoge_key.pub hoge@hoge_ip:~/.ssh/<br>
        <br>
        # 作業サーバーで実行<br>
        cat .ssh/hoge_key.pub >> .ssh/authorized_keys<br>
        rm .ssh/hoge_key.pub<br>
        </div>
    4. 権限設定
        <br>
        <br>
        <div style="position: relative; background-color:#002b36; color:#ffffff; padding:10px; border-radius:5px; font-family:monospace;">
        <div style="position:absolute; top:-10px; left:10px; background:#268bd2; color:#fff; padding:2px 6px; border-radius:3px; font-size:12px; font-family:sans-serif;">
            bash
        </div>
        chmod 700 .ssh<br>
        chmod 600 .ssh/authorized_keys<br>
        </div>

---

### **3. conda 環境の作成**
情科演習の作業で使う仮想環境のセットアップを行います．
1. WEBからダウンロードリンクをコピーしてくる  
    Annaconda URL:https://www.anaconda.com/ 
<table>
  <tr>
    <td style="text-align: center;">
      <img src="../images/anaconda1.png" width="500" alt="画像1の説明">
    </td>
    <td style="text-align: center;">
      <img src="../images/anaconda2.png" width="500" alt="画像2の説明">
    </td>
  </tr>
</table>

2. wgetコマンドでダウンロード
        <br>
        <br>
        <div style="position: relative; background-color:#002b36; color:#ffffff; padding:10px; border-radius:5px; font-family:monospace;">
        <div style="position:absolute; top:-10px; left:10px; background:#268bd2; color:#fff; padding:2px 6px; border-radius:3px; font-size:12px; font-family:sans-serif;">
            bash
        </div>
        wget https://repo.anaconda.com/archive/Anaconda3-2025.06-0-Linux-x86_64.sh<br>
        </div>
3. インストール  
    sudoを使うとroot権限になってユーザーから見えなくなるので注意
        <br>
        <br>
        <div style="position: relative; background-color:#002b36; color:#ffffff; padding:10px; border-radius:5px; font-family:monospace;">
        <div style="position:absolute; top:-10px; left:10px; background:#268bd2; color:#fff; padding:2px 6px; border-radius:3px; font-size:12px; font-family:sans-serif;">
            bash
        </div>
        bash ./Anaconda3-2025.06-0-Linux-x86_64.sh -b -p ~/anaconda3<br>
        </div>
4. 初期化してパス通す  
        <br>
        <br>
        <div style="position: relative; background-color:#002b36; color:#ffffff; padding:10px; border-radius:5px; font-family:monospace;">
        <div style="position:absolute; top:-10px; left:10px; background:#268bd2; color:#fff; padding:2px 6px; border-radius:3px; font-size:12px; font-family:sans-serif;">
            bash
        </div>
        ~/anaconda3/bin/conda init bash
        <br>
        source ~/.bashrc<br>
        </div>
6. 仮想環境作成  
    初回は利用規約確認が出るが無視してOK
        <br>
        <br>
        <div style="position: relative; background-color:#002b36; color:#ffffff; padding:10px; border-radius:5px; font-family:monospace;">
        <div style="position:absolute; top:-10px; left:10px; background:#268bd2; color:#fff; padding:2px 6px; border-radius:3px; font-size:12px; font-family:sans-serif;">
            bash
        </div>
        # 仮想環境作成  
        conda create -n env_test python=3.9<br>
        # 仮想環境有効化  
        conda activate env_test<br>
        </div>
7. その他使うコマンド
        <br>
        <br>
        <div style="position: relative; background-color:#002b36; color:#ffffff; padding:10px; border-radius:5px; font-family:monospace;">
        <div style="position:absolute; top:-10px; left:10px; background:#268bd2; color:#fff; padding:2px 6px; border-radius:3px; font-size:12px; font-family:sans-serif;">
            bash
        </div>
        # 仮想環境作終了(baseに戻る)  
        conda deactivate<br>
        # 環境一覧表示  
        conda env list<br>
        # パッケージのインストール  
        conda install numpy pandas<br>
        # パッケージ一覧表示  
        conda list<br>
        # 環境削除  
        conda remove -n env_test --all<br>
        # 環境のエクスポート  
        conda env export > env.yml<br>
        # 環境の再構築  
        conda env create -f env.yml<br>
        </div>

---

### **4. GPU周りの設定**
作業サーバーに積んであるGPUを使えるようにします．
1. GPU ドライバー設定
    1. CUDA対応のGPUが刺さっているか確認  
    今回はGeForce RTX 3060 Ti
        <br>
        <br>
        <div style="position: relative; background-color:#002b36; color:#ffffff; padding:10px; border-radius:5px; font-family:monospace;">
        <div style="position:absolute; top:-10px; left:10px; background:#268bd2; color:#fff; padding:2px 6px; border-radius:3px; font-size:12px; font-family:sans-serif;">
            bash
        </div>
        lspci | grep -i nvidia<br>
        </div>
    2. 公式サイト(https://www.nvidia.com/en-us/drivers/)から，使用しているGPUに対応するドライバーを探す  
    ※ubuntu-drivers devicesでも実は探せる
        <table>
        <tr>
            <td style="text-align: center;">
            <img src="../images/driver1.png" width="500" alt="画像1の説明">
            </td>
            <td style="text-align: center;">
            <img src="../images/driver2.png" width="500" alt="画像2の説明">
            </td>
        </tr>
        </table>
        <img src="../images/driver3.png" width="1050"> 
        <br>
    3. ドライバインストール
        <br>
        <br>
        <div style="position: relative; background-color:#002b36; color:#ffffff; padding:10px; border-radius:5px; font-family:monospace;">
        <div style="position:absolute; top:-10px; left:10px; background:#268bd2; color:#fff; padding:2px 6px; border-radius:3px; font-size:12px; font-family:sans-serif;">
            bash
        </div>
        sudo apt update<br> 
        sudo apt install nvidia-driver-580<br>
        </div>
2. CUDA設定
    1. GPUに合うCUDA verの確認  
    CUDAのwikipedia(https://en.wikipedia.org/wiki/CUDA#GPUs_supported)で確認
        1. 3060tiのMicro architectureはAmpere  
        2. Compute capabilityは8.6  
        3. Compute capabilityに対応するCUDA verは11.1-11.4  
            公式テストしたver．安定．怖かったらこれ．  
        4. Micro architectureに対応するCUDA verは11.0-13.0  
            全然動作する  
            11.1-11.4が最適だが古いので今回は12.6を使用 
        <table>
        <tr>
            <td style="text-align: center;">
            <img src="../images/cuda1.png" width="500">
            </td>
            <td style="text-align: center;">
            <img src="../images/cuda2.png" width="500">
            </td>
        </tr>
        </table> 
    2. CUDA 12.6のインストール  
    公式サイトから12.6のコードをコピー  
        <img src="../images/cuda3.png" width="1050"> 
        <br>
        <br>
        <br>
        <div style="position: relative; background-color:#002b36; color:#ffffff; padding:10px; border-radius:5px; font-family:monospace;">
        <div style="position:absolute; top:-10px; left:10px; background:#268bd2; color:#fff; padding:2px 6px; border-radius:3px; font-size:12px; font-family:sans-serif;">
            bash
        </div> 
        wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-keyring_1.1-1_all.deb<br> 
        sudo dpkg -i cuda-keyring_1.1-1_all.deb<br> 
        sudo apt-get update<br> 
        sudo apt-get -y install cuda-toolkit-12-6<br> 
        </div>
    4. 確認
        <br>
        <br>
        <div style="position: relative; background-color:#002b36; color:#ffffff; padding:10px; border-radius:5px; font-family:monospace;">
        <div style="position:absolute; top:-10px; left:10px; background:#268bd2; color:#fff; padding:2px 6px; border-radius:3px; font-size:12px; font-family:sans-serif;">
            bash
        </div> 
        nvcc --version<br> 
        </div>

---

### **5. PyTorchのインストール**
画像パート機械学習編で使うPyTorchの準備をします  
1. CUDA verと対応するPyTorchのverを確認する  
    PyTorch公式(https://pytorch.org/get-started/locally/)  
    <img src="../images/pytorch1.png" width="1050"> 
    <br>
2. 仮想環境上でコードを実行  
    <br>
    <br>
    <div style="position: relative; background-color:#002b36; color:#ffffff; padding:10px; border-radius:5px; font-family:monospace;">
    <div style="position:absolute; top:-10px; left:10px; background:#268bd2; color:#fff; padding:2px 6px; border-radius:3px; font-size:12px; font-family:sans-serif;">
        bash
    </div>
    pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cu126<br>
    </div>
3. 確認
    <br>
    <br>
    <div style="position: relative; background-color:#002b36; color:#ffffff; padding:10px; border-radius:5px; font-family:monospace;">
    <div style="position:absolute; top:-10px; left:10px; background:#268bd2; color:#fff; padding:2px 6px; border-radius:3px; font-size:12px; font-family:sans-serif;">
        bash
    </div>
    python3 -c "import torch; print('CUDA available:', torch.cuda.is_available()); print('GPU:', torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None')"<br>
    </div>

---
### **セットアップ完了！**